In [61]:
import os
import cohere
import pandas as pd
import random
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
co = cohere.Client('API KEY')

In [63]:
file_path1 = r"D:\Machine translation and summarization model\Summerization model\Random samples of test data splits\randomsampling bengali\urls_and_articles_summary_bengali.csv"
file_path2 = r"D:\Machine translation and summarization model\Summerization model\Random samples of test data splits\randomsampling gujarati\urls_and_articles_summary_gujarati.csv"
file_path3 = r"D:\Machine translation and summarization model\Summerization model\Random samples of test data splits\randomsampling hindi\urls_and_articles_summary_hindi.csv"
file_path4 = r"D:\Machine translation and summarization model\Summerization model\Random samples of test data splits\randomsampling marathi\urls_and_articles_summary_marathi.csv"
file_path5 = r"D:\Machine translation and summarization model\Summerization model\Random samples of test data splits\randomsampling tamil\urls_and_articles_summary_tamil.csv"
file_path6 = r"D:\Machine translation and summarization model\Summerization model\Random samples of test data splits\randomsampling telugu\urls_and_articles_summary_telugu.csv"

# Load each file and store in a separate DataFrame
df1 = pd.read_csv(file_path1, encoding='utf-8')
df2 = pd.read_csv(file_path2, encoding='utf-8')
df3 = pd.read_csv(file_path3, encoding='utf-8')
df4 = pd.read_csv(file_path4, encoding='utf-8')
df5 = pd.read_csv(file_path5, encoding='utf-8')
df6 = pd.read_csv(file_path6, encoding='utf-8')

# Preview the data for each file to confirm the "Article" column is loaded correctly
print("File 1 Preview:")
print(df1.head())

print("File 2 Preview:")
print(df2.head())

print("File 3 Preview:")
print(df3.head())

print("File 4 Preview:")
print(df4.head())

print("File 5 Preview:")
print(df5.head())

print("File 6 Preview:")
print(df6.head())

File 1 Preview:
                                  URL Source  \
0  https://www.bbc.com/bengali/news-56713767   
1  https://www.bbc.com/bengali/news-56533637   
2  https://www.bbc.com/bengali/news-56631353   
3  https://www.bbc.com/bengali/news-47189627   
4  https://www.bbc.com/bengali/news-43949363   

                                             Article  \
0  বাংলাদেশের ইসলামিক ফাউন্ডেশনও গত ১৪ই মার্চ ইসল...   
1  বিক্ষোভের সময় পুলিশ কাঁদানে গ্যাস ও রাবার বুল...   
2  ডুবে যাওয়া লঞ্চটি মুন্সিগঞ্জ থেকে নারায়ণগঞ্জ...   
3  ২০১৮ সালের নির্বাচন প্রত্যাখ্যান করেছে বিএনপি।...   
4  কার্টুনে দাম্পত্য জীবন মহারাষ্ট্রের কোলাপুরের ...   

                                             Summary  
0  ইসলামী শিক্ষাবিদ এবং যুক্তরাজ্যের জাতীয় স্বাস...  
1  বাংলাদেশে ভারতীয় প্রধানমন্ত্রী নরেন্দ্র মোদীর...  
2  বাংলাদেশে রাজধানী ঢাকার নিকটবর্তী নারায়ণগঞ্জে...  
3  বাংলাদেশে গত ৩০শে ডিসেম্বর অনুষ্ঠিত সংসদ নির্ব...  
4  বিয়ে টিকিয়ে রাখার ক্ষেত্রে স্বামী ও স্ত্রীর ...  
File 2 Preview:
            

In [122]:
def get_last_full_stop(text):
    """Helper function to find the last full stop or '।' in the summary."""
    """Ensure the summary ends with a full stop or '।'."""
    # First, try to find the last full stop (.)
    last_full_stop = max(text.rfind('.'), text.rfind('।'))
    if last_full_stop != -1:
        return text[:last_full_stop + 1]
    return text.strip() + '।' 

def summarize_article(article, target_language, max_tokens=150):
    try:
        # Define the conversation-like structure
        messages = [
            {"role": "user", "content": f"Summarize the following article in {target_language}. Please ensure the summary is complete, concise, and includes all the key points."},
            {"role": "assistant", "content": f"Article:\n{article}"}
        ]
        
        # Construct the prompt from the conversation
        prompt = "\n".join([f"{message['role']}: {message['content']}" for message in messages])
        
        # Generate the summary
        response = co.generate(
            model='command-r-plus',  # Cohere's text generation model
            prompt=prompt,
            max_tokens=max_tokens,   # Max token length for summary
            temperature= random.uniform(0.6, 1.0),  # Random temperature for creativity
            stop_sequences = [] 
        )
        
        summary = response.generations[0].text.strip()

        
        summary = get_last_full_stop(summary)

        return summary

    except Exception as e:
        print(f"Error summarizing article: {e}")
        return None

In [67]:
def process_dataframe(df, target_language):
    all_summaries = []

    # Iterate over each article in the DataFrame
    for index, article in enumerate(df['Article']):
        summary = summarize_article(article, target_language)
        all_summaries.append(summary)

        # Print the generated summary for the current article in real-time
        print(f"Processing Article {index + 1}/{len(df)}:")
        print(f"Article: {article[:100]}...")  # Print the first 100 characters of the article for context
        print(f"Generated Summary: {summary}")
        print("-" * 100)  # Separator for clarity

    df['Generated_Summary'] = all_summaries
    return df

In [71]:
target_language = "Hindi"
print("Processing articles and generating summaries...")
processed_df = process_dataframe(df3,target_language)
output_file_path = "D:\Machine translation and summarization model\Summerization model\OUTPUT SUMMARIES\COHERE\Cohere_summaries_Hindi.csv" 
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
processed_df.to_csv(output_file_path, index=False, encoding='utf-8-sig') 
print(f"Results saved to {output_file_path}")

Processing articles and generating summaries...
Processing Article 1/150:
Article: दोनों ने भारत के वीनू मनकड और पंकज राय का 52 साल पुराना रिकॉर्ड तोड़ दिया. स्मिथ और मैकेंज़ी ने चटगा...
Generated Summary: दक्षिण अफ्रीका के ग्रीम स्मिथ और नील मैकेंज़ी ने पहले विकेट के लिए 415 रन की साझेदारी कर एक नया विश्व रिकॉर्ड कायम किया है. इन दोनों ने भारत के वीनू मनकड़ और पंकज राय का 52 साल पुराना रिकॉर्ड तोड़ दिया है.
----------------------------------------------------------------------------------------------------
Processing Article 2/150:
Article: ये खबर इंडियन एक्सप्रेस के पहले पन्ने पर है. पार्टी ने कहा कि अगर वो पश्चिम बंगाल में जीतती है तो वह...
Generated Summary: यहाँ भारतीय समाचार पत्रों से ली गई कुछ प्रमुख खबरें दी गई हैं:

- भारतीय जनता पार्टी ने कहा कि अगर वह पश्चिम बंगाल में जीतती है, तो राज्य में राष्ट्रीय नागरिक पंजी (एनआरसी) लागू करेगी.
----------------------------------------------------------------------------------------------------
Processing Article 3/150:
Article: माना जा 

In [80]:
target_language = "Bengali"
print("Processing articles and generating summaries...")
processed_df = process_dataframe(df1,target_language)
output_file_path = "D:\Machine translation and summarization model\Summerization model\OUTPUT SUMMARIES\COHERE\Cohere_summaries_Bengali.csv" 
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
processed_df.to_csv(output_file_path, index=False, encoding='utf-8-sig') 
print(f"Results saved to {output_file_path}")

Processing articles and generating summaries...
Processing Article 1/150:
Article: বাংলাদেশের ইসলামিক ফাউন্ডেশনও গত ১৪ই মার্চ ইসলামিক ফাউন্ডেশনের সভা কক্ষে দেশের জ্যেষ্ঠ আলেমদের সঙ্গে...
Generated Summary: ইসলামিক ফাউন্ডেশন এবং দেশের জ্যেষ্ঠ আলেমরা বলছেন, রমজান মাসে রোজা রাখার সময় করোনাভাইরাসের টিকা নেয়া কোনও কারণেই রোজা ভঙ্গ হবে না।
----------------------------------------------------------------------------------------------------
Processing Article 2/150:
Article: বিক্ষোভের সময় পুলিশ কাঁদানে গ্যাস ও রাবার বুলেট ছোঁড়ে, বিক্ষোভকারীরাও ইটপাটকেল নিক্ষেপ করে। জানা গ...
Generated Summary: ঢাকায় ভারতের প্রধানমন্ত্রী নরেন্দ্র মোদীর সফরের প্রতিবাদে বায়তুল মোকাররম এলাকায় শুক্রবার ঘটেছে বিক্ষোভ ও সংঘর্ষ।
----------------------------------------------------------------------------------------------------
Processing Article 3/150:
Article: ডুবে যাওয়া লঞ্চটি মুন্সিগঞ্জ থেকে নারায়ণগঞ্জের উদ্দেশ্যে রওনা হয়েছিল। (ফাইল ছবি) নৌ-পুলিশের নারায...
Generated Summary: একটি মালবাহী জাহাজের সাথে ধা

In [114]:
target_language = "Marathi"
print("Processing articles and generating summaries...")
processed_df = process_dataframe(df4,target_language)
output_file_path = "D:\Machine translation and summarization model\Summerization model\OUTPUT SUMMARIES\COHERE\COHERE_summaries_Marathi.csv" 
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
processed_df.to_csv(output_file_path, index=False, encoding='utf-8-sig') 
print(f"Results saved to {output_file_path}")

Processing articles and generating summaries...
Processing Article 1/150:
Article: लवकर केस पांढरे होत असतील तर त्याला वैद्यकीय भाषेत याला केनाइटिस असं म्हणतात. वयाच्या 20 वर्षांआधीचं...
Generated Summary: केसाचे पांढरापेट होणे कारण वैद्यकीय भाषेत केनाइटिस म्हणतात. हे वयाच्या 20 वर्षांपूर्वीच होऊ शकते आणि केसात रंगवर्धक पदार्थाची कमी म्हणजे मेलानिनची कमी होते.
----------------------------------------------------------------------------------------------------
Processing Article 2/150:
Article: स्टीव्हन स्मिथने वेगवान शतक झळकावलं. सिडनी क्रिकेट ग्राऊंडवर सुरू असलेल्या या मॅचमध्ये कोरोना कारणास...
Generated Summary: सिडनी क्रिकेट ग्राउंडवर भारत व ऑस्ट्रेलिया मध्येच्या दुसऱ्या एकदिवसीय मैचात स्टीव्ह स्मिथ यांनी शतक मारले.
----------------------------------------------------------------------------------------------------
Processing Article 3/150:
Article: कोरोना लसीकरण खुद्द पंतप्रधान मोदींनीही 1 मार्चला, म्हणजे दुसऱ्या टप्प्याच्या लसीकरणाच्या पहिल्याच द...
Generated Summary: पंतप्रधान मो

In [120]:
target_language = "Gujarati"
print("Processing articles and generating summaries...")
processed_df = process_dataframe(df2,target_language)
output_file_path = "D:\Machine translation and summarization model\Summerization model\OUTPUT SUMMARIES\COHERE\COHERE_with_summaries_Gujarati.csv" 
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
processed_df.to_csv(output_file_path, index=False, encoding='utf-8-sig') 
print(f"Results saved to {output_file_path}")

Processing articles and generating summaries...
Processing Article 1/150:
Article: જોકે, રાજપૂતાના રાઇફલ્સની બીજી બટાલિયનમાં જોડાયેલા લેફ્ટનન્ટ હિતેશ કુમારના પિતા લાન્સ નાયક બચન સિંહ ...
Generated Summary: હિતેશ કુમાર, જોડાયેલા લેફ્ટનન્ટ, રાજપૂતાના રાઇફલ્સની બીજી બટાલિયનમાં જોડાયાં છે.
----------------------------------------------------------------------------------------------------
Processing Article 2/150:
Article: સામાચાર એજન્સી એએનઆઈ પ્રમાણે આગ લાગવાની આ ઘટના શૉર્ટ સર્કિટના કારણે થઈ છે. જોકે, અધિકારીઓના જણાવ્યા ...
Generated Summary: સામાચાર એજન્સી એએનઆઈએ કહે છે કે ઘટના શૉર્ટ સર્કિટના કારણે થઈ છે.
----------------------------------------------------------------------------------------------------
Processing Article 3/150:
Article: શશી થરૂરની ટિપ્પણીથી સોશિયલ મીડિયામાં ચર્ચા થરૂરે 'બેંગલુરુ લિટરેચર ફૅસ્ટ'માં આરએસએસના એક અજાણ્યા સૂ...
Generated Summary: શશી થરૂરે બેંગલુરુમાં એક કાર્યક્રમમાં વડા પ્રધાનિ નરેન્દ્ર મોદીની તુલના આપતાં વિવાદાસ્।
------------------------------------------

In [124]:
target_language = "Tamil"
print("Processing articles and generating summaries...")
processed_df = process_dataframe(df5,target_language)
output_file_path = "D:\Machine translation and summarization model\Summerization model\OUTPUT SUMMARIES\COHERE\COHERE_summaries_Tamil.csv" 
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
processed_df.to_csv(output_file_path, index=False, encoding='utf-8-sig') 
print(f"Results saved to {output_file_path}")

Processing articles and generating summaries...
Processing Article 1/150:
Article: தீர்ப்பு வெளியான பிறகு செய்தியாளர்களிடம் பேசிய மு.க. ஸ்டாலின், "ஜனநாயகத்தை காக்கக்கூடிய வகையில் இந்த...
Generated Summary: தி.மு.க தலைவர் மு.க. ஸ்டாலின் கூறுகையில், உள்ளாட்சித் தேர்தல் தொடர்பான உச்ச நீதிமன்றத் தீர்ப்பை தி.மு.
----------------------------------------------------------------------------------------------------
Processing Article 2/150:
Article: சித்தரிப்புக்காக வெனிசுவேலா நாட்டில் ஏறத்தாழ 50 ஆயிரம் வாக்கு இயந்திரங்கள் எரிந்து சாம்பலானதாக அந்நா...
Generated Summary: வெனிசுவேலாவில் சித்தரிப்புக்காக ஏறத்தாழ 50 ஆயிரம் வாக்கு இயந்திரங்கள் எரிந்து சாம்பலானதால், இந்த ஆண்டு இறுதியில் நடைபெறவிருந்த தேர்தல்।
----------------------------------------------------------------------------------------------------
Processing Article 3/150:
Article: எங்களின் போராட்டம் மக்களின் எதார்த்த வாழ்க்கையை பாதிக்காது என்றும், அரசியலில் ரஜினிகாந்தை எதிர்க்கும...
Generated Summary: தினத்தந்தி செய்தித் தாள், கமல்ஹாசன் த

In [125]:
target_language = "Telugu"
print("Processing articles and generating summaries...")
processed_df = process_dataframe(df6,target_language)
output_file_path = "D:\Machine translation and summarization model\Summerization model\OUTPUT SUMMARIES\COHERE\COHERE_summaries_Telugu.csv" 
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
processed_df.to_csv(output_file_path, index=False, encoding='utf-8-sig') 
print(f"Results saved to {output_file_path}")

Processing articles and generating summaries...
Processing Article 1/150:
Article: ప్రధాని నరేంద్ర మోదీ ప్రైమ్ మినిస్టర్స్ సిటిజన్ అసిస్టెన్స్ అండ్ రిలీఫ్ ఇన్ ఎమర్జెన్సీ సిచువేషన్(పీఎ...
Generated Summary: సుప్రీం కోర్టు ప్రధాని నరేంద్ర మోదీ కేర్ ఫండ్ నిధిని నేషనల్ డిజాస్టర్ రిలీఫ్ ఫండ్ (ఎన్డిఆర్ఎఫ్)।
----------------------------------------------------------------------------------------------------
Processing Article 2/150:
Article: 2020 అధ్యక్ష ఎన్నికల్లో జోక్యం చేసుకోవడానికి ట్విటర్ ప్రయత్నిస్తోందని ట్రంప్ ఆరోపించారు ''దేర్‌ ఈజ్‌...
Generated Summary: డోనాల్డ్ ట్రంప్ అమెరికా అధ్యక్షుడుగా 2020 అధ్యక్ష ఎన్నికల్లో తలదూర్చేందుకు ట్విటర్ ప్రయత్నిస్�।
----------------------------------------------------------------------------------------------------
Processing Article 3/150:
Article: సుప్రీంకోర్టు నలుగురు సుప్రీంకోర్టు సీనియర్ న్యాయమూర్తులు లేవనెత్తిన అంశాలపై సుప్రీంకోర్టు న్యాయమూర్...
Generated Summary: నలుగురు సుప్రీంకోర్టు న్యాయమూర్తులు సీనియర్ న్యాయమూర్తులతో చర్చించిన అంశాల గురించి 